In [1]:
from numba import jit
import numpy as np
import awkward.forth
import awkward as ak

In [126]:
machine = awkward.forth.ForthMachine32("1 if 10 else 40 then 20") 
machine.run()
machine.stack

[10, 20]

In [127]:
bytecodes = ak.Array(
    ak.contents.ListOffsetArray(
        ak.index.Index64(machine.bytecodes_offsets),
    ak.contents.NumpyArray(machine.bytecodes),
    )
)
bytecodes

<Array [[0, 1, 4, 72, 73, 0, 20], [...], [0, 40]] type='3 * var * int64'>

In [121]:
def compile(bytecodes):
    pycode = []
    num_indent = 1
    indent = "    "
    for i in range(len(bytecodes)):
        if i == 0:
            pycode.extend(["@jit(nopython=True, nogil=True)\n", "def func():\n", "    cur = np.uint64(0)\n", "    stack = np.zeros(100, dtype=np.int64)\n"])
        else:
            if i == 1:
                pycode.append("    print(stack, cur)\n")
            pycode.extend(["@jit(nopython=True, nogil=True)\n", f"def func_{i}(stack,cur):\n"])
        cursor = 0
        code = bytecodes[i]
        while cursor < len(code):
            if code[cursor] == 0:#push
                cursor += 1
                pycode.append(indent*num_indent+f"cur = stack_push(stack, {code[cursor]}, cur)\n")
                cursor += 1
            elif code[cursor] == 3: #if-then
                pycode.append(indent*num_indent+f"temp, cur = stack_pop(stack, cur)\n")
                pycode.append(indent*num_indent+f"if temp != 0:\n")
                num_indent += 1
                cursor += 1
                func_address = code[cursor]-71
                pycode.append(indent*num_indent+f"stack, cur = func_{func_address}(stack,cur)\n")
                num_indent -= 1
                cursor += 1
            elif code[cursor] == 4:#if-else-then
                pycode.append(indent*num_indent+f"temp, cur = stack_pop(stack, cur)\n")
                pycode.append(indent*num_indent+f"if temp != 0:\n")
                num_indent += 1
                cursor += 1
                func_address = code[cursor]-71
                pycode.append(indent*num_indent+f"stack, cur = func_{func_address}(stack,cur)\n")
                num_indent -= 1
                pycode.append(indent*num_indent+f"else:\n")
                cursor += 1
                num_indent += 1
                func_address = code[cursor]-71
                pycode.append(indent*num_indent+f"stack, cur = func_{func_address}(stack, cur)\n")
                num_indent -= 1
                cursor += 1
            elif code[cursor] == 38:#pop
                pycode.append(indent*num_indent+f"temp, cur = stack_pop(stack, cur)\n")
                cursor += 1
            elif code[cursor] == 44:#add
                pycode.append(indent*num_indent+f"temp1,cur = stack_pop(stack, cur)\n")
                pycode.append(indent*num_indent+f"temp2, cur = stack_pop(stack, cur)\n")
                pycode.append(indent*num_indent+f"cur = stack_push(stack, temp1+temp2, cur)\n")
                cursor += 1
            elif code[cursor] == 45:#subtract
                pycode.append(indent*num_indent+f"temp1,cur = stack_pop(stack, cur)\n")
                pycode.append(indent*num_indent+f"temp2, cur = stack_pop(stack, cur)\n")
                pycode.append(indent*num_indent+f"cur = stack_push(stack, temp2-temp1, cur)\n")
                cursor += 1
        if i >= 1:
            pycode.append("    return stack, cur\n")
    return pycode

In [128]:
pycode = compile(ak.to_list(bytecodes))
pycode.append("func()")
aa = "".join(pycode)

In [129]:
@jit(nopython=True, nogil=True)
def stack_push(stack,elem, cur):
    if cur == len(stack)-1:
        raise ValueError("Index out of range")
    stack[cur] = elem
    cur += 1
    return cur

@jit(nopython=True, nogil=True)
def stack_pop(stack, cur):
    if cur == 0:
        raise ValueError("Index out of range")
    cur -= 1
    return stack[cur], cur


In [130]:
print(aa)

@jit(nopython=True, nogil=True)
def func():
    cur = np.uint64(0)
    stack = np.zeros(100, dtype=np.int64)
    cur = stack_push(stack, 1, cur)
    temp, cur = stack_pop(stack, cur)
    if temp != 0:
        stack, cur = func_1(stack,cur)
    else:
        stack, cur = func_2(stack, cur)
    cur = stack_push(stack, 20, cur)
    print(stack, cur)
@jit(nopython=True, nogil=True)
def func_1(stack,cur):
    cur = stack_push(stack, 10, cur)
    return stack, cur
@jit(nopython=True, nogil=True)
def func_2(stack,cur):
    cur = stack_push(stack, 40, cur)
    return stack, cur
func()


In [131]:
exec(aa)

[10 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0] 2


In [49]:
aa

'@jit(nopython=True, nogil=True)\ndef func():\ncur = np.uint64(0)\nstack = np.zeros(100, dtype=np.int64)\n    cur = stack.push(stack, 10, cur)\n    temp, cur = stack.pop(stack, cur)\\]\n    cur = stack.push(stack, 10, cur)\n    cur = stack.push(stack, 10, cur)\n    temp1,cur = stack.pop(stack, cur)\n    temp2, cur = stack.pop(stack, cur)\n    cur = stack.push(stack, temp1+temp2, cur)\n'